<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%B0%B1%ED%85%8C%EC%8A%A4%ED%8A%B8_%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_FAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [ ]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 24.6 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
# start_date = today - timedelta(days=375)

# 전략명
str_strategy = 'FAA '

# 전략에 사용할 ETF 명 리스트에 담기
etf_stock = ['VTI', 'VEA', 'VWO']
etf_bond = ['SHY', 'BND']
etf_real = ['GSG', 'VNQ']
etf_cash = ['SHY']

# 자산군 리스트 더하기, 중복 제거
etf_order = list(set(etf_stock + etf_bond + etf_real + etf_cash))

# ETF 가격 추출
df_etf = yf.download(etf_order, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0).dropna()

display(df_etf)
display(df_etf[:-4*MBD:MBD])

[*********************100%%**********************]  7 of 7 completed


,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,45.5000,222.8100,70.8900,22.4000,40.6600,80.9300,81.5900
2023-09-08,44.9900,221.4200,71.0000,22.2500,40.2000,80.9100,81.5800
2023-09-07,45.0500,221.1800,70.9800,22.1700,40.1200,80.9500,81.9000
2023-09-06,45.2300,221.9400,70.7400,22.2100,40.5400,80.8000,81.3900
2023-09-05,45.4100,223.3400,70.8200,22.1400,40.7300,80.8900,81.5600
...,...,...,...,...,...,...,...
2007-08-01,29.7174,53.5226,45.7364,44.1800,30.0923,65.1052,33.6378
2007-07-31,29.5938,53.2088,45.7585,44.5200,30.5776,65.1685,33.1991
2007-07-30,29.6309,53.9472,45.6667,44.1500,31.0144,65.0634,33.3318
2007-07-27,29.1428,53.1607,45.7095,44.1500,29.8658,65.1443,32.6993


,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,45.5000,222.8100,70.8900,22.4000,40.6600,80.9300,81.5900
2023-08-10,46.3800,222.0100,71.2748,21.5900,41.4300,80.8445,83.2000
2023-07-12,46.7700,222.4000,72.0794,20.2700,41.7300,80.7327,85.7500
2023-06-09,45.6879,213.1168,71.9763,19.3200,40.6388,80.6566,81.7568
2023-05-10,45.9356,204.1909,73.0488,19.4700,40.0023,81.4322,82.0238
...,...,...,...,...,...,...,...
2008-04-04,28.9584,50.8534,49.1803,59.6700,32.1857,69.7057,34.8713
2008-03-05,27.6659,49.4538,48.4605,59.5100,32.3703,69.7096,31.2740
2008-02-04,28.1269,50.9861,49.0724,52.0000,32.0011,69.1354,32.9346
2008-01-03,30.2424,53.4094,48.5275,54.6600,34.6154,68.1591,31.0959


In [ ]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} backtest {str_strategy}.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')


# 월평균영업일수. average Monthly Business Day
MBD = 21
'''
# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '4mo_ago/' + str(df_etf.index[MBD*4].date()), \
                     '6mo_ago/' + str(df_etf.index[MBD*6].date()), '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-4-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_4mo = df_etf.shift(-MBD*4).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_4mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-4-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)
'''

'\n# 인덱스 만들기\ndf_momentum_index = [\'today/\' + str(df_etf.index[0].date()), \'1mo_ago/\' + str(df_etf.index[MBD].date()),                      \'3mo_ago/\' + str(df_etf.index[MBD*3].date()), \'4mo_ago/\' + str(df_etf.index[MBD*4].date()),                      \'6mo_ago/\' + str(df_etf.index[MBD*6].date()), \'12mo_ago/\' + str(df_etf.index[MBD*12].date())]\n\n\n## DataFrame 만들기: 1-3-4-6-12 개월 etf 가격\nprice_today = df_etf.iloc[0:1]\nprice_1mo = df_etf.shift(-MBD).iloc[0:1]\nprice_3mo = df_etf.shift(-MBD*3).iloc[0:1]\nprice_4mo = df_etf.shift(-MBD*4).iloc[0:1]\nprice_6mo = df_etf.shift(-MBD*6).iloc[0:1]\nprice_12mo = df_etf.shift(-MBD*12).iloc[0:1]\n\ndf_price = pd.concat([price_today, price_1mo, price_3mo, price_4mo, price_6mo, price_12mo])\ndf_price.index = df_momentum_index\n\nstr_etf_price = str_today + str_strategy + "### ETF 현재-1-3-4-6-12 개월 가격"\ndisplay(Markdown(str_etf_price))\ndisplay(df_price)\n\n# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함\nwith open(output_path + filename, \'w\')

In [ ]:
## DataFrame 만들기: 수익률
ror_4mo = df_etf.pct_change(-MBD*4)*100   # 4개월 수익률
df_etf_ror = ror_4mo.dropna()[::MBD]

str_etf_ror = str_today + str_strategy + "### ETF 4 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

## DataFrame 만들기: 수익률 점수
df_ror_score = df_etf_ror.rank(ascending=False, method='max', axis=1) * 10
display(df_ror_score)
'''
# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)
'''

### 2023-09-12 FAA ### ETF 4 개월 수익률

,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,-0.9483,9.1185,-2.9553,15.0488,1.6441,-0.6167,-0.5289
2023-08-10,2.7178,9.3134,-2.3892,3.7482,2.3476,-0.2357,1.2265
2023-07-12,8.6688,15.9820,0.7283,0.6955,7.3299,0.8778,8.3870
2023-06-09,2.1708,3.9394,-0.2977,-6.5764,-0.8978,0.8233,-7.8677
2023-05-10,6.4615,6.0142,1.3319,-3.6616,-2.3317,1.6574,-0.7329
...,...,...,...,...,...,...,...
2008-04-04,-6.6304,-6.1094,1.8104,19.0780,-7.4871,2.6927,3.5601
2008-03-05,-12.2831,-11.0977,2.2001,15.8909,-11.5860,4.4372,-12.7452
2008-02-04,-9.7072,-9.9343,4.7866,11.7318,-6.4870,4.3363,-13.8357
2008-01-03,1.7878,-2.2152,4.5650,25.4245,11.6572,3.5085,-13.4001


,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,60.0000,20.0000,70.0000,10.0000,30.0000,50.0000,40.0000
2023-08-10,30.0000,10.0000,70.0000,20.0000,40.0000,60.0000,50.0000
2023-07-12,20.0000,10.0000,60.0000,70.0000,40.0000,50.0000,30.0000
2023-06-09,20.0000,10.0000,40.0000,60.0000,50.0000,30.0000,70.0000
2023-05-10,10.0000,20.0000,40.0000,70.0000,60.0000,30.0000,50.0000
...,...,...,...,...,...,...,...
2008-04-04,60.0000,50.0000,40.0000,10.0000,70.0000,30.0000,20.0000
2008-03-05,60.0000,40.0000,30.0000,10.0000,50.0000,20.0000,70.0000
2008-02-04,50.0000,60.0000,20.0000,10.0000,40.0000,30.0000,70.0000
2008-01-03,50.0000,60.0000,30.0000,10.0000,20.0000,40.0000,70.0000


"\n# csv에 제목 및 DataFrame 추가\nwith open(output_path + filename, 'a') as f:\n  f.write('\n')\n  f.write(f'{str_etf_ror}\n')\n  df_etf_ror.to_csv(f, index=True)\n"

In [ ]:
## DataFrame 만들기: 변동성

# 4개월 변동성 계산
etf_volatility_raw = df_etf[etf_order].pct_change().rolling(window=MBD*4).std() * (252**0.5)
etf_volatility = etf_volatility_raw[::MBD].shift(-4).dropna()

# 변동성 데이터를 출력
str_etf_volatility = str_today + str_strategy + "### 4개월 변동성 (연간화)"

display(Markdown(str_etf_volatility))
display(etf_volatility)


## DataFrame 만들기: 변동성 점수
etf_volatility_score = etf_volatility.rank(ascending=False, method='max', axis=1) * 5
display(etf_volatility_score)

'''
# 변동성 데이터를 CSV 파일에 추가
with open(output_path + filename, 'a') as f:
    f.write('\n')
    f.write(f'{str_etf_volatility}\n')
    df_volatility.to_csv(f, index=True)
'''

### 2023-09-12 FAA ### 4개월 변동성 (연간화)

,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,0.1340,0.1092,0.0611,0.1717,0.1523,0.0196,0.1498
2023-08-10,0.1329,0.1158,0.0669,0.1983,0.1505,0.0235,0.1622
2023-07-12,0.1465,0.1293,0.0721,0.2074,0.1414,0.0360,0.1877
2023-06-09,0.1443,0.1465,0.0748,0.1999,0.1417,0.0372,0.2057
2023-05-10,0.1367,0.1589,0.0785,0.2078,0.1353,0.0374,0.2119
...,...,...,...,...,...,...,...
2008-04-04,0.2407,0.2308,0.0668,0.2985,0.3724,0.0291,0.4006
2008-03-05,0.2389,0.2137,0.0619,0.2473,0.3869,0.0267,0.3827
2008-02-04,0.2309,0.2108,0.0528,0.2380,0.4018,0.0265,0.3823
2008-01-03,0.2142,0.1840,0.0518,0.2235,0.3850,0.0261,0.3390


,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,20.0000,25.0000,30.0000,5.0000,10.0000,35.0000,15.0000
2023-08-10,20.0000,25.0000,30.0000,5.0000,15.0000,35.0000,10.0000
2023-07-12,15.0000,25.0000,30.0000,5.0000,20.0000,35.0000,10.0000
2023-06-09,20.0000,15.0000,30.0000,10.0000,25.0000,35.0000,5.0000
2023-05-10,20.0000,15.0000,30.0000,10.0000,25.0000,35.0000,5.0000
...,...,...,...,...,...,...,...
2008-04-04,20.0000,25.0000,30.0000,15.0000,10.0000,35.0000,5.0000
2008-03-05,20.0000,25.0000,30.0000,15.0000,5.0000,35.0000,10.0000
2008-02-04,20.0000,25.0000,30.0000,15.0000,5.0000,35.0000,10.0000
2008-01-03,20.0000,25.0000,30.0000,15.0000,5.0000,35.0000,10.0000


"\n# 변동성 데이터를 CSV 파일에 추가\nwith open(output_path + filename, 'a') as f:\n    f.write('\n')\n    f.write(f'{str_etf_volatility}\n')\n    df_volatility.to_csv(f, index=True)\n"

In [ ]:
## DataFrame 만들기: 상관성

# 4개월 변동성 계산
etf_corr_raw = df_etf[etf_order]  # len -> 4059

etf_corr = pd.DataFrame()

for corr_i in range(0, len(df_etf)-MBD*4, MBD):
  etf_corr_sum = pd.DataFrame(etf_corr_raw[corr_i:corr_i + MBD*4].corr().sum(axis=1) - 1).T  # 절대값 사용
  etf_corr = pd.concat([etf_corr, etf_corr_sum])

etf_corr.index = etf_volatility_score.index

display(etf_corr)


## DataFrame 만들기: 상관성 점수
etf_corr_score = etf_corr.rank(ascending=False, method='max', axis=1) * 5
display(etf_corr_score)


'''
# 변동성 데이터를 CSV 파일에 추가
with open(output_path + filename, 'a') as f:
    f.write('\n')
    f.write(f'{str_etf_volatility}\n')
    df_volatility.to_csv(f, index=True)
'''

,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,2.8206,2.2766,-0.0342,1.2076,2.8650,-0.7025,2.6727
2023-08-10,2.7941,1.7140,-0.8661,2.0824,2.4963,-1.1100,2.4447
2023-07-12,2.1976,1.0332,0.2702,1.2856,2.1281,-0.6146,1.7328
2023-06-09,2.3467,1.8102,0.7550,0.5999,2.1860,0.1968,0.3921
2023-05-10,2.5625,2.1521,1.3895,1.4411,2.2123,-0.1669,1.2134
...,...,...,...,...,...,...,...
2008-04-04,0.2359,0.0051,-1.5171,-0.4170,0.1169,-1.1726,0.7088
2008-03-05,0.4424,0.4559,-2.0869,-0.6612,0.6678,-1.8074,0.3402
2008-02-04,0.4945,0.2880,-1.8540,-0.8266,0.6385,-1.8485,-0.1982
2008-01-03,2.2544,0.4788,0.6703,1.2853,2.6405,0.5568,-0.6463


,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,10.0000,20.0000,30.0000,25.0000,5.0000,35.0000,15.0000
2023-08-10,5.0000,25.0000,30.0000,20.0000,10.0000,35.0000,15.0000
2023-07-12,5.0000,25.0000,30.0000,20.0000,10.0000,35.0000,15.0000
2023-06-09,5.0000,15.0000,20.0000,25.0000,10.0000,35.0000,30.0000
2023-05-10,5.0000,15.0000,25.0000,20.0000,10.0000,35.0000,30.0000
...,...,...,...,...,...,...,...
2008-04-04,10.0000,20.0000,35.0000,25.0000,15.0000,30.0000,5.0000
2008-03-05,15.0000,10.0000,35.0000,25.0000,5.0000,30.0000,20.0000
2008-02-04,10.0000,15.0000,35.0000,25.0000,5.0000,30.0000,20.0000
2008-01-03,10.0000,30.0000,20.0000,15.0000,5.0000,25.0000,35.0000


"\n# 변동성 데이터를 CSV 파일에 추가\nwith open(output_path + filename, 'a') as f:\n    f.write('\n')\n    f.write(f'{str_etf_volatility}\n')\n    df_volatility.to_csv(f, index=True)\n"

In [ ]:
## 수익률 점수(100%) + 변동성 점수(50%) + 상관성 점수(50%)
etf_total_score = df_ror_score + etf_volatility_score + etf_corr_score
display(etf_total_score)

,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,90.0000,65.0000,130.0000,40.0000,45.0000,120.0000,70.0000
2023-08-10,55.0000,60.0000,130.0000,45.0000,65.0000,130.0000,75.0000
2023-07-12,40.0000,60.0000,120.0000,95.0000,70.0000,120.0000,55.0000
2023-06-09,45.0000,40.0000,90.0000,95.0000,85.0000,100.0000,105.0000
2023-05-10,35.0000,50.0000,95.0000,100.0000,95.0000,100.0000,85.0000
...,...,...,...,...,...,...,...
2008-04-04,90.0000,95.0000,105.0000,50.0000,95.0000,95.0000,30.0000
2008-03-05,95.0000,75.0000,95.0000,50.0000,60.0000,85.0000,100.0000
2008-02-04,80.0000,100.0000,85.0000,50.0000,50.0000,95.0000,100.0000
2008-01-03,80.0000,115.0000,80.0000,40.0000,30.0000,100.0000,115.0000


In [ ]:
monthly_index = df_etf_ror.index

top3 = etf_total_score.apply(lambda row: row.nlargest(3).index.tolist(), axis=1)
df_top3_etf = pd.DataFrame(top3.to_list(), columns=['1st', '2nd', '3rd'], index=monthly_index)
display(df_top3_etf)

etf_ror_list = []
row_count = len(top3)

for i in range(row_count):
  a_list = df_etf_ror.iloc[i][df_top3.iloc[i]].tolist()
  etf_ror_list.append(a_list)

df_top3_ror = pd.DataFrame(etf_ror_list, columns=['1st', '2nd', '3rd'], index=monthly_index)
display(df_top3_ror)

# 수익률 0 미만은 SHY로 교체
for i in range(row_count):
  for j in range(3):
    if df_top3_ror.iloc[i][j] < 0:
      df_top3_etf.iloc[i][j] = 'SHY'

display(df_top3_etf)
# SHY 가 매번 너무 많고, SHY 아닌것도 -수익률이 너무 많아서 투자 않기로

,1st,2nd,3rd
2023-09-11,BND,SHY,VEA
2023-08-10,BND,SHY,VNQ
2023-07-12,BND,SHY,GSG
2023-06-09,VNQ,SHY,GSG
2023-05-10,GSG,SHY,BND
...,...,...,...
2008-04-04,BND,VTI,VWO
2008-03-05,VNQ,VEA,BND
2008-02-04,VTI,VNQ,SHY
2008-01-03,VTI,VNQ,SHY


,1st,2nd,3rd
2023-09-11,-2.9553,-0.6167,-0.9483
2023-08-10,-2.3892,-0.2357,1.2265
2023-07-12,0.7283,0.8778,0.6955
2023-06-09,-7.8677,0.8233,-6.5764
2023-05-10,-3.6616,1.6574,1.3319
...,...,...,...
2008-04-04,1.8104,-6.1094,-7.4871
2008-03-05,-12.7452,-12.2831,2.2001
2008-02-04,-9.9343,-13.8357,4.3363
2008-01-03,-2.2152,-13.4001,3.5085


,1st,2nd,3rd
2023-09-11,SHY,SHY,SHY
2023-08-10,SHY,SHY,VNQ
2023-07-12,BND,SHY,GSG
2023-06-09,SHY,SHY,SHY
2023-05-10,SHY,SHY,BND
...,...,...,...
2008-04-04,BND,SHY,SHY
2008-03-05,SHY,SHY,BND
2008-02-04,SHY,SHY,SHY
2008-01-03,SHY,SHY,SHY


In [ ]:
df_etf_monthly = df_etf[:-4*MBD:MBD]
display(df_etf_monthly)

monthly_row_count = df_top3_etf.shape[0]
monthly_column_count = df_top3_etf.shape[1]

df_buy_ratio = pd.DataFrame(np.full((monthly_row_count, monthly_column_count), 1/3), columns=['1st', '2nd', '3rd'], index=df_etf_ror.index)
display(df_buy_ratio)

,VEA,VTI,BND,GSG,VWO,SHY,VNQ
2023-09-11,45.5000,222.8100,70.8900,22.4000,40.6600,80.9300,81.5900
2023-08-10,46.3800,222.0100,71.2748,21.5900,41.4300,80.8445,83.2000
2023-07-12,46.7700,222.4000,72.0794,20.2700,41.7300,80.7327,85.7500
2023-06-09,45.6879,213.1168,71.9763,19.3200,40.6388,80.6566,81.7568
2023-05-10,45.9356,204.1909,73.0488,19.4700,40.0023,81.4322,82.0238
...,...,...,...,...,...,...,...
2008-04-04,28.9584,50.8534,49.1803,59.6700,32.1857,69.7057,34.8713
2008-03-05,27.6659,49.4538,48.4606,59.5100,32.3703,69.7096,31.2740
2008-02-04,28.1269,50.9861,49.0725,52.0000,32.0011,69.1354,32.9346
2008-01-03,30.2424,53.4094,48.5275,54.6600,34.6154,68.1591,31.0959


,1st,2nd,3rd
2023-09-11,0.3333,0.3333,0.3333
2023-08-10,0.3333,0.3333,0.3333
2023-07-12,0.3333,0.3333,0.3333
2023-06-09,0.3333,0.3333,0.3333
2023-05-10,0.3333,0.3333,0.3333
...,...,...,...
2008-04-04,0.3333,0.3333,0.3333
2008-03-05,0.3333,0.3333,0.3333
2008-02-04,0.3333,0.3333,0.3333
2008-01-03,0.3333,0.3333,0.3333


In [ ]:
# 비율대로 돈 투자
df_invest_money = pd.DataFrame(columns=['1st', '2nd', '3rd'], index=monthly_index)
df_invest_count = pd.DataFrame(columns=['1st', '2nd', '3rd'], index=monthly_index)
df_cash = pd.DataFrame(columns=['cash'], index=monthly_index)

# 초기자본
invest_cash = 10_000

for i in reversed(range(row_count)):
  row_sum = 0

  for j in range(3):
    if i == row_count - 1:
      row_sum = invest_cash
      break
    else:
      sel_etf = df_top3_etf.iloc[i+1][j]
      row_sum += df_invest_count.iloc[i+1][j] * df_etf_monthly.iloc[i][sel_etf]

  for j in range(3):
    df_cash.iloc[i][0] = row_sum
    sel_etf = df_top3_etf.iloc[i][j]
    sel_etf_price = df_etf_monthly.iloc[i][sel_etf]
    sel_etf_buy_count = df_cash.iloc[i][0] * df_buy_ratio.iloc[i][j] / sel_etf_price
    df_invest_count.iloc[i][j] = sel_etf_buy_count

display(Markdown('### etf 구매 개수'))
display(df_invest_count)
display(Markdown('### 평가 금액'))
display(df_cash)

CAGR = str(round(((df_cash.iloc[0][0] / df_cash.iloc[-1][0]) ** (12/(row_count-1)) - 1)*100, 2))
display(Markdown(f'## CAGR: {CAGR} %'))

### etf 구매 개수

,1st,2nd,3rd
2023-09-11,53.5472,53.5472,53.5472
2023-08-10,53.9135,53.9135,52.3872
2023-07-12,59.3743,53.0103,211.1329
2023-06-09,53.0103,53.0103,53.0103
2023-05-10,53.1025,53.1025,59.1967
...,...,...,...
2008-04-04,67.6567,47.7347,47.7347
2008-03-05,47.4986,47.4986,68.3258
2008-02-04,47.4986,47.4986,47.4986
2008-01-03,47.4986,47.4986,47.4986


### 평가 금액

,cash
2023-09-11,13000.7159
2023-08-10,13075.8381
2023-07-12,12838.9929
2023-06-09,12826.8877
2023-05-10,12972.7470
...,...
2008-04-04,9982.1329
2008-03-05,9933.3247
2008-02-04,9851.5096
2008-01-03,9712.3882


## CAGR: 1.68 %

In [ ]:
'''
# SPY, EFA 수익이 둘 다 BIL 보다 낮으면 채권에 투자
str_is_bond = 'False'
if ror_12mo['SPY'].iloc[0] < ror_12mo['BIL'].iloc[0] and ror_12mo['EFA'].iloc[0] < ror_12mo['BIL'].iloc[0]:
  str_is_bond = 'True'

# 채권 중 수익률 상위 3개에 각 33.3% 씩 투자! 그 중 6개월 수익률이 -이면 현금 보유
str_bond_top_3 = str_today + str_strategy + '### 6개월 수익률 안전자산 상위 3개'
ma_6mo_defensive_top_3 = ror_6mo[etf_defensive].transpose().nlargest(3, str(df_etf.index[0].date())).transpose()
display(Markdown(str_bond_top_3))
display(ma_6mo_defensive_top_3)

# 안전자산 중 0 이상인 것만 필터링하기
defensive_etf_list_text = ''

if str_is_bond == 'True':
  filtered_index = ma_6mo_defensive_top_3.columns[(ma_6mo_defensive_top_3 < 0).any()]
  df_etf_defensive_positive_index = ma_6mo_defensive_top_3.drop(filtered_index, axis=1).columns
  defensive_etf_list_text = ' '.join([str(asset) for asset in df_etf_defensive_positive_index])


## 결과 출력
display(Markdown(str_today[1:] + str_strategy))

str_etf = "## 12개월 수익률 기준, 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? " + str_is_bond
display(Markdown(str_etf))

# 아니라면 SPY, EFA 중 12개월 수익률이 더 높은 것 표시
str_higher_stock = ''

if str_is_bond == 'False':
  if ror_12mo['SPY'].iloc[0] > ror_12mo['EFA'].iloc[0]:
    str_higher_stock = 'SPY'
  else:
    str_higher_stock = 'EFA'

str_higher_stock_text = "## 투자할 공격자산: " + str_higher_stock
display(Markdown(str_higher_stock_text))

# 안전자산 중 6개월 수익률 0 이상 출력
str_etf_defensive_positive = '## 투자할 안전자산: ' + defensive_etf_list_text
display(Markdown(str_etf_defensive_positive))

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_etf}\n')
  f.write(f'{str_higher_stock_text}\n')
  f.write(f'{str_etf_defensive_positive}\n')
'''

'\n# SPY, EFA 수익이 둘 다 BIL 보다 낮으면 채권에 투자\nstr_is_bond = \'False\'\nif ror_12mo[\'SPY\'].iloc[0] < ror_12mo[\'BIL\'].iloc[0] and ror_12mo[\'EFA\'].iloc[0] < ror_12mo[\'BIL\'].iloc[0]:\n  str_is_bond = \'True\'\n\n# 채권 중 수익률 상위 3개에 각 33.3% 씩 투자! 그 중 6개월 수익률이 -이면 현금 보유\nstr_bond_top_3 = str_today + str_strategy + \'### 6개월 수익률 안전자산 상위 3개\'\nma_6mo_defensive_top_3 = ror_6mo[etf_defensive].transpose().nlargest(3, str(df_etf.index[0].date())).transpose()\ndisplay(Markdown(str_bond_top_3))\ndisplay(ma_6mo_defensive_top_3)\n\n# 안전자산 중 0 이상인 것만 필터링하기\ndefensive_etf_list_text = \'\'\n\nif str_is_bond == \'True\':\n  filtered_index = ma_6mo_defensive_top_3.columns[(ma_6mo_defensive_top_3 < 0).any()]\n  df_etf_defensive_positive_index = ma_6mo_defensive_top_3.drop(filtered_index, axis=1).columns\n  defensive_etf_list_text = \' \'.join([str(asset) for asset in df_etf_defensive_positive_index])\n\n\n## 결과 출력\ndisplay(Markdown(str_today[1:] + str_strategy))\n\nstr_etf = "## 12개월 수익률 기준, 미국 주식(SPY)과 글로